In [1]:
import pandas as pd

In [2]:
df_ans = pd.read_csv("../preprocess_test_data.csv",index_col=0)
df_ans['text_clean'] = df_ans['text_clean'].fillna('neutral')
df_ans.head()

,_id,text,task_1,task_2,task_3,task_4,text_clean
0,Hindi_image_410.jpg,"Sign You are Bancho a] _ ~""11|7 have best ffen...",Sarcastic,Positive,Vulgar,Non-abusive,"Sign You are Bancho a] _ ~""11|7 have best ffen..."
1,Hindi_image_114.jpg,एक महिला घोडे़ के लिंग लिया| घोड़ा उत्साहित हो...,Non-Sarcastic,Negative,Vulgar,Abusive,एक महिला घोडे़ के लिंग लिया| घोड़ा उत्साहित हो...
2,Hindi_image_101.jpg,एक टीचर ने एक लड़के को पेपर में नक़ल करते पकड लि...,Non-Sarcastic,Neutral,Non Vulgar,Non-abusive,एक टीचर ने एक लड़के को पेपर में नक़ल करते पकड लि...
3,Hindi_image_1747.jpg,show me Sckht Launda Kisslay Jha CTrollerlzabu...,Sarcastic,Negative,Vulgar,Abusive,show me Sckht Launda Kisslay Jha CTrollerlzabu...
4,Hindi_image_19.jpg,पति सुहागरात में पत्नी की निप्पल चूसते हुए बोल...,Non-Sarcastic,Negative,Non Vulgar,Abusive,पति सुहागरात में पत्नी की निप्पल चूसते हुए बोल...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=2)
X_test = tfidfconverter.fit_transform(df_ans['text_clean']).toarray()

In [4]:
import pickle
# 加载模型和特征转换器
with open('vectorizer.pkl', 'rb') as f:
    tfidfconverter = pickle.load(f)
X_test = tfidfconverter.transform(df_ans['text_clean']).toarray()

# task_1

In [5]:
import pickle
kn1= pickle.load(open('best_KN_model1.sav', 'rb'))
lr1= pickle.load(open('best_LR_model1.sav', 'rb'))
nb1= pickle.load(open('best_NB_model1.sav', 'rb'))
rf1= pickle.load(open('best_RF_model1.sav', 'rb'))
svm1= pickle.load(open('best_SVM_model1.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Sarcastic'):
            res[i] = 0
        else:
            res[i] = 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn1.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf1.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm1.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(58, 119, 164)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Hindi_image_410.jpg
1,Hindi_image_114.jpg
2,Hindi_image_101.jpg
3,Hindi_image_1747.jpg
4,Hindi_image_19.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,0
1,Hindi_image_114.jpg,0
2,Hindi_image_101.jpg,0
3,Hindi_image_1747.jpg,0
4,Hindi_image_19.jpg,0
5,Hindi_image_1917.jpg,0
6,Hindi_image_179.jpg,0
7,Hindi_image_27.jpg,0
8,Hindi_image_181.jpg,0
9,Hindi_image_107.jpg,0


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,0
2,Hindi_image_101.jpg,0
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,1
5,Hindi_image_1917.jpg,0
6,Hindi_image_179.jpg,0
7,Hindi_image_27.jpg,0
8,Hindi_image_181.jpg,0
9,Hindi_image_107.jpg,0


In [13]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [14]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,0,0,1,0
1,0,0,0,0
2,0,0,0,0
3,0,0,1,0
4,0,0,1,0


In [15]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Sarcastic'
    else:
        res[i] = 'Non-Sarcastic'

In [16]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic


In [17]:
df_ans.to_csv('tfidf_overall.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic
5,Hindi_image_1917.jpg,Sarcastic
6,Hindi_image_179.jpg,Sarcastic
7,Hindi_image_27.jpg,Sarcastic
8,Hindi_image_181.jpg,Sarcastic
9,Hindi_image_107.jpg,Sarcastic


In [18]:
res= pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i]='Sarcastic'
    else:
        res[i]='Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic


In [19]:
df_ans.to_csv('tfidf_rft.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic
5,Hindi_image_1917.jpg,Sarcastic
6,Hindi_image_179.jpg,Sarcastic
7,Hindi_image_27.jpg,Sarcastic
8,Hindi_image_181.jpg,Sarcastic
9,Hindi_image_107.jpg,Sarcastic


In [20]:
res = pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i]='Sarcastic'
    else:
        res[i]='Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non-Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Non-Sarcastic
4,Hindi_image_19.jpg,Non-Sarcastic


In [21]:
df_ans.to_csv('tfidf_svm.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non-Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Non-Sarcastic
4,Hindi_image_19.jpg,Non-Sarcastic
5,Hindi_image_1917.jpg,Sarcastic
6,Hindi_image_179.jpg,Sarcastic
7,Hindi_image_27.jpg,Sarcastic
8,Hindi_image_181.jpg,Sarcastic
9,Hindi_image_107.jpg,Sarcastic


In [22]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i]='Sarcastic'
    else:
        res[i]='Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic


In [23]:
df_ans.to_csv('tfidf_knc.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Sarcastic
1,Hindi_image_114.jpg,Sarcastic
2,Hindi_image_101.jpg,Sarcastic
3,Hindi_image_1747.jpg,Sarcastic
4,Hindi_image_19.jpg,Sarcastic
5,Hindi_image_1917.jpg,Sarcastic
6,Hindi_image_179.jpg,Sarcastic
7,Hindi_image_27.jpg,Sarcastic
8,Hindi_image_181.jpg,Sarcastic
9,Hindi_image_107.jpg,Non-Sarcastic


# task_3

In [5]:
import pickle
kn3= pickle.load(open('best_KN_model3.sav', 'rb'))
lr3= pickle.load(open('best_LR_model3.sav', 'rb'))
nb3= pickle.load(open('best_NB_model3.sav', 'rb'))
rf3= pickle.load(open('best_RF_model3.sav', 'rb'))
svm3= pickle.load(open('best_SVM_model3.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Vulgar'):
            res[i]= 0
        else:
            res[i]= 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn3.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf3.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm3.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(737, 613, 587)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Hindi_image_410.jpg
1,Hindi_image_114.jpg
2,Hindi_image_101.jpg
3,Hindi_image_1747.jpg
4,Hindi_image_19.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,0
2,Hindi_image_101.jpg,0
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,0
5,Hindi_image_1917.jpg,1
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,0
8,Hindi_image_181.jpg,0
9,Hindi_image_107.jpg,1


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,0
2,Hindi_image_101.jpg,1
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,0
5,Hindi_image_1917.jpg,1
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,0
8,Hindi_image_181.jpg,0
9,Hindi_image_107.jpg,1


In [13]:
df_ans['label'] = pd.Series(pred_kn)
df_ans.to_csv('tfidf_knc3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,1
2,Hindi_image_101.jpg,1
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,0
5,Hindi_image_1917.jpg,1
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,1
8,Hindi_image_181.jpg,1
9,Hindi_image_107.jpg,1


In [14]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [15]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,1,1,1,1
1,1,0,0,0
2,1,0,1,1
3,1,1,1,1
4,0,0,0,0


In [16]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'

In [17]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar


In [18]:
df_ans.to_csv('tfidf_overall3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar
5,Hindi_image_1917.jpg,Non Vulgar
6,Hindi_image_179.jpg,Non Vulgar
7,Hindi_image_27.jpg,Vulgar
8,Hindi_image_181.jpg,Vulgar
9,Hindi_image_107.jpg,Non Vulgar


In [19]:
res = pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar


In [20]:
df_ans.to_csv('tfidf_rft3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar
5,Hindi_image_1917.jpg,Non Vulgar
6,Hindi_image_179.jpg,Non Vulgar
7,Hindi_image_27.jpg,Vulgar
8,Hindi_image_181.jpg,Vulgar
9,Hindi_image_107.jpg,Non Vulgar


In [21]:
res = pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar


In [22]:
df_ans.to_csv('tfidf_svm3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar
5,Hindi_image_1917.jpg,Non Vulgar
6,Hindi_image_179.jpg,Non Vulgar
7,Hindi_image_27.jpg,Vulgar
8,Hindi_image_181.jpg,Vulgar
9,Hindi_image_107.jpg,Non Vulgar


In [23]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Non Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar


In [24]:
df_ans.to_csv('tfidf_knc3.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non Vulgar
1,Hindi_image_114.jpg,Non Vulgar
2,Hindi_image_101.jpg,Non Vulgar
3,Hindi_image_1747.jpg,Non Vulgar
4,Hindi_image_19.jpg,Vulgar
5,Hindi_image_1917.jpg,Non Vulgar
6,Hindi_image_179.jpg,Non Vulgar
7,Hindi_image_27.jpg,Non Vulgar
8,Hindi_image_181.jpg,Non Vulgar
9,Hindi_image_107.jpg,Non Vulgar


# task_4

In [5]:
import pickle
kn4= pickle.load(open('best_KN_model4.sav', 'rb'))
lr4= pickle.load(open('best_LR_model4.sav', 'rb'))
nb4= pickle.load(open('best_NB_model4.sav', 'rb'))
rf4= pickle.load(open('best_RF_model4.sav', 'rb'))
svm4= pickle.load(open('best_SVM_model4.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Abusive'):
            res[i]= 0
        else:
            res[i]= 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn4.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf4.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm4.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(745, 689, 657)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Hindi_image_410.jpg
1,Hindi_image_114.jpg
2,Hindi_image_101.jpg
3,Hindi_image_1747.jpg
4,Hindi_image_19.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,1
2,Hindi_image_101.jpg,1
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,1
5,Hindi_image_1917.jpg,1
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,1
8,Hindi_image_181.jpg,1
9,Hindi_image_107.jpg,1


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,1
2,Hindi_image_101.jpg,1
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,1
5,Hindi_image_1917.jpg,0
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,1
8,Hindi_image_181.jpg,1
9,Hindi_image_107.jpg,1


In [13]:
df_ans['label'] = pd.Series(pred_kn)
df_ans.to_csv('tfidf_knc4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,1
1,Hindi_image_114.jpg,1
2,Hindi_image_101.jpg,1
3,Hindi_image_1747.jpg,1
4,Hindi_image_19.jpg,1
5,Hindi_image_1917.jpg,1
6,Hindi_image_179.jpg,1
7,Hindi_image_27.jpg,1
8,Hindi_image_181.jpg,1
9,Hindi_image_107.jpg,1


In [14]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [15]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1


In [16]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'

In [17]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive


In [18]:
df_ans.to_csv('tfidf_overall4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive
5,Hindi_image_1917.jpg,Non-abusive
6,Hindi_image_179.jpg,Non-abusive
7,Hindi_image_27.jpg,Non-abusive
8,Hindi_image_181.jpg,Non-abusive
9,Hindi_image_107.jpg,Non-abusive


In [19]:
res = pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive


In [20]:
df_ans.to_csv('tfidf_rft4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive
5,Hindi_image_1917.jpg,Non-abusive
6,Hindi_image_179.jpg,Non-abusive
7,Hindi_image_27.jpg,Non-abusive
8,Hindi_image_181.jpg,Non-abusive
9,Hindi_image_107.jpg,Non-abusive


In [21]:
res = pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive


In [22]:
df_ans.to_csv('tfidf_svm4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive
5,Hindi_image_1917.jpg,Abusive
6,Hindi_image_179.jpg,Non-abusive
7,Hindi_image_27.jpg,Non-abusive
8,Hindi_image_181.jpg,Non-abusive
9,Hindi_image_107.jpg,Non-abusive


In [23]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive


In [24]:
df_ans.to_csv('tfidf_knc4.csv',index=False)
df_ans.head(20)

,_id,label
0,Hindi_image_410.jpg,Non-abusive
1,Hindi_image_114.jpg,Non-abusive
2,Hindi_image_101.jpg,Non-abusive
3,Hindi_image_1747.jpg,Non-abusive
4,Hindi_image_19.jpg,Non-abusive
5,Hindi_image_1917.jpg,Non-abusive
6,Hindi_image_179.jpg,Non-abusive
7,Hindi_image_27.jpg,Non-abusive
8,Hindi_image_181.jpg,Non-abusive
9,Hindi_image_107.jpg,Non-abusive
